In [3]:
import re
#import csv
import pandas as pd

from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from nltk.classify.scikitlearn import SklearnClassifier

In [90]:
#pandas read csv
#read data
train_df = pd.read_csv("/Users/ClaireWang/Documents/study/previous/STOR 767/train.csv",header = 0, encoding = 'ISO-8859-1')
test_df = pd.read_csv("/Users/ClaireWang/Documents/study/previous/STOR 767/test.csv",header = 0, encoding = 'ISO-8859-1')
#train_df.head(5)
#test_df.head(5)

train_df.columns = ['text','sentiment']
test_df.columns = ['text','sentiment']

In [77]:
#data processing - tokenization, stop words removal, non-letters removal, stemming

#Pandas DataFrame columns are Pandas Series when you pull them out, 
#which you can then call .tolist() on to turn them into a Python list
train_text = train_df.text.tolist()
test_text = test_df.text.tolist()

train_sentiment = train_df.sentiment
test_sentiment = test_df.sentiment

import nltk
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()

def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    #remove non letters
    text = re.sub('[^a-zA-Z0-9]',' ',text)
    #tokenize
    tokens = nltk.word_tokenize(text)
    #stem
    stems = stem_tokens(tokens,stemmer)
    return stems

for i in range(len(train_text)):
    train_text[i] = train_text[i].lstrip('"').rstrip().lower()
    train_text[i] = ' '.join(tokenize(train_text[i]))


In [76]:
# #f read
# #read data
# with open("/Users/ClaireWang/Documents/study/previous/STOR 767/train.csv",encoding="ISO-8859-1") as f:
#     f_train = f.readlines()[1:]
# with open("/Users/ClaireWang/Documents/study/previous/STOR 767/test.csv",encoding="ISO-8859-1") as f:
#     f_test = f.readlines()[1:]


In [78]:
#bag-of-words
#CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

vectorizer = CountVectorizer(
    analyzer = 'word',
    tokenizer = tokenize,
    lowercase = True,
    stop_words = 'english',
    min_df=5
    #max_df=0.8
)

In [107]:
#tfidf vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer_t = TfidfVectorizer(
    min_df=5,
    stop_words = 'english',
    lowercase = True,
    tokenizer = tokenize,
    analyzer='word',
    sublinear_tf = True
    #max_df = 0.8
)

In [98]:
#transform data into feature vectors & fit the model
corpus = train_text
corpus2 = test_text 

#The method fit_transform does two functions: First, it fits the model and learns the vocabulary; 
#second, it transforms our corpus data into feature vectors. 
#The input to fit_transform should be a list of strings, so we concatenate train and test data as follows.

#CountVectorizer
corpus_data_features = vectorizer.fit_transform(corpus)
corpus_data_features2 = vectorizer.transform(corpus2)
#Numpy arrays are easy to work with, so convert the result to an array.
corpus_data_features_nd = corpus_data_features.toarray()
corpus_data_features_nd2 = corpus_data_features2.toarray()

In [108]:
#TfIDF
corpus_data_features_t = vectorizer_t.fit_transform(corpus)
corpus_data_features2_t = vectorizer_t.transform(corpus2)
#Numpy arrays are easy to work with, so convert the result to an array.
corpus_data_features_nd_t = corpus_data_features_t.toarray()
corpus_data_features_nd2_t = corpus_data_features2_t.toarray()

In [85]:
#check features and the counts
#vectorizer._validate_vocabulary()
vocab = vectorizer.get_feature_names()
# Sum up the counts of each vocabulary word
dist = np.sum(corpus_data_features_nd, axis=0)
    
# For each, print the vocabulary word and the number of times it 
# appears in the data set
#for tag, count in zip(vocab, dist):
    #print(count, tag)

17 0
5 00
23 000
1 00am
2 01
2 02
2 03
1 04
2 05
2 07
1 08
1 0ne
169 1
347 10
33 100
6 1000
2 10000000000000
1 1000lb
1 100x
7 101
2 102
2 104
2 108
1 10line
1 10p
1 10pm
2 10th
31 11
1 116
20 12
1 120
1 12383499143743701
1 12th
20 13
6 13th
18 14
1 1415
3 14th
37 15
1 1561
11 16
1 163
1 16mm
1 16th
15 17
1 1700
2 1780
1 17million
19 18
5 180
2 1800
1 1809
2 1813
1 1820
1 1837
1 1852
1 1861
1 1863
2 1865
1 1880
1 1881
1 1886
1 188o
3 1890
1 1896
1 1898
2 18th
4 19
3 1900
1 1901
2 1910
4 1913
5 1914
2 1915
2 1918
1 1919
6 1920
2 1922
1 1924
2 1926
1 1927
1 1929
17 1930
3 1931
8 1932
10 1933
5 1934
2 1935
3 1936
1 1937
4 1938
7 1939
1 193o
3 1940
2 1941
4 1942
7 1943
3 1944
4 1945
1 1946
4 1947
6 1948
4 1949
38 1950
4 1951
4 1952
2 1953
2 1954
6 1955
1 1956
2 1957
5 1958
4 1959
10 1960
3 1961
5 1962
5 1963
8 1964
7 1965
2 1966
4 1967
5 1968
5 1969
20 1970
11 1971
5 1972
9 1973
3 1974
4 1975
5 1976
6 1977
7 1978
12 1979
1 197o
22 1980
5 1981
7 1982
12 1983
8 1984
6 1985
2 1986
1 1987
5 19

In [102]:
#Logistic Regression - CountVectorizer
#train classifer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import time

log_model = LogisticRegression()
t0=time.time()
log_model = log_model.fit(X=corpus_data_features_nd,y=train_sentiment)
t1=time.time()
#label evaluation set.We can use either predict for classes or predict_proba for probabilities.
y_pred = log_model.predict(corpus_data_features_nd2)
t2=time.time()
time_lr_train = t1-t0
time_lr_predict = t2-t1

#There is a function for classification called sklearn.metrics.classification_report
#which calculates several types of (predictive) scores on a classification model.

print("Results for Logistic Regression - CountVectorizer")
print("Training time: %fs; Prediction time: %fs" % (time_lr_train, time_lr_predict))
print(classification_report(test_sentiment,y_pred))

Results for Logistic Regression
Training time: 0.553749s; Prediction time: 0.255698s
             precision    recall  f1-score   support

   negative       0.83      0.84      0.84      1000
   positive       0.84      0.83      0.83      1000

avg / total       0.84      0.84      0.84      2000



In [109]:
#Logistic Regression - TFIDF Vectorizer
#train classifer
log_model = LogisticRegression()
t0=time.time()
log_model = log_model.fit(X=corpus_data_features_nd_t,y=train_sentiment)
t1=time.time()
#label evaluation set.We can use either predict for classes or predict_proba for probabilities.
y_pred = log_model.predict(corpus_data_features_nd2_t)
t2=time.time()
time_lr_train = t1-t0
time_lr_predict = t2-t1

#There is a function for classification called sklearn.metrics.classification_report
#which calculates several types of (predictive) scores on a classification model.

print("Results for Logistic Regression - TFIDF")
print("Training time: %fs; Prediction time: %fs" % (time_lr_train, time_lr_predict))
print(classification_report(test_sentiment,y_pred))

Results for Logistic Regression - TFIDF
Training time: 0.085751s; Prediction time: 0.025347s
             precision    recall  f1-score   support

   negative       0.86      0.83      0.84      1000
   positive       0.84      0.86      0.85      1000

avg / total       0.85      0.85      0.85      2000



In [110]:
#SVM, kernel=RBF - CountVectorizer
from sklearn import svm

CountV_train = corpus_data_features_nd
CountV_test = corpus_data_features_nd2

#SVM with kernel = RBF
classifier_rbf = svm.SVC()
t0=time.time()
classifier_rbf.fit(CountV_train,train_df.sentiment)
t1=time.time()
prediction_rbf = classifier_rbf.predict(CountV_test)
t2=time.time()
time_rbf_train = t1-t0
time_rbf_predict = t2-t1

#Print results
print("Results for SVC(kernel=rbf) - CountVectorizer")
print("Training time: %fs; Prediction time: %fs" % (time_rbf_train, time_rbf_predict))
print(classification_report(test_sentiment, prediction_rbf))

Results for SVC(kernel=rbf) - CountVectorizer
Training time: 94.763433s; Prediction time: 93.294180s
             precision    recall  f1-score   support

   negative       0.76      0.33      0.46      1000
   positive       0.57      0.90      0.70      1000

avg / total       0.67      0.61      0.58      2000



In [111]:
#SVM, kernel=RBF - TFIDF Vectorizer
CountV_train_t = corpus_data_features_nd_t
CountV_test_t = corpus_data_features_nd2_t

#SVM with kernel = RBF
classifier_rbf = svm.SVC()
t0=time.time()
classifier_rbf.fit(CountV_train_t,train_df.sentiment)
t1=time.time()
prediction_rbf = classifier_rbf.predict(CountV_test_t)
t2=time.time()
time_rbf_train = t1-t0
time_rbf_predict = t2-t1

#Print results
print("Results for SVC(kernel=rbf) - TFIDF")
print("Training time: %fs; Prediction time: %fs" % (time_rbf_train, time_rbf_predict))
print(classification_report(test_sentiment, prediction_rbf))

Results for SVC(kernel=rbf) - TFIDF
Training time: 24.631071s; Prediction time: 25.738746s
             precision    recall  f1-score   support

   negative       0.93      0.55      0.69      1000
   positive       0.68      0.96      0.79      1000

avg / total       0.80      0.75      0.74      2000



In [103]:
#SVM with kernel = linear 1 
classifier_linear = svm.SVC(kernel='linear')
t0=time.time()
classifier_linear.fit(CountV_train,train_df.sentiment)
t1=time.time()
prediction_linear = classifier_linear.predict(CountV_test)
t2=time.time()
time_linear_train = t1-t0
time_linear_predict = t2-t1

print("Results for SVC(kernel=linear)")
print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))
print(classification_report(test_sentiment, prediction_linear))

Results for SVC(kernel=linear)
Training time: 74.177153s; Prediction time: 49.359496s
             precision    recall  f1-score   support

   negative       0.81      0.84      0.82      1000
   positive       0.83      0.81      0.82      1000

avg / total       0.82      0.82      0.82      2000



In [112]:
#SVM with kernel = linear 1 - TFIDF
classifier_linear = svm.SVC(kernel='linear')
t0=time.time()
classifier_linear.fit(CountV_train_t,train_df.sentiment)
t1=time.time()
prediction_linear = classifier_linear.predict(CountV_test_t)
t2=time.time()
time_linear_train = t1-t0
time_linear_predict = t2-t1

print("Results for SVC(kernel=linear) - TFIDF")
print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))
print(classification_report(test_sentiment, prediction_linear))

Results for SVC(kernel=linear) - TFIDF
Training time: 19.961402s; Prediction time: 15.943797s
             precision    recall  f1-score   support

   negative       0.85      0.85      0.85      1000
   positive       0.85      0.85      0.85      1000

avg / total       0.85      0.85      0.85      2000



In [104]:
#SVM with kernel = linear 2
classifier_liblinear = svm.LinearSVC()
t0=time.time()
classifier_liblinear.fit(CountV_train,train_df.sentiment)
t1=time.time()
prediction_liblinear = classifier_liblinear.predict(CountV_test)
t2=time.time()
time_liblinear_train = t1-t0
time_liblinear_predict = t2-t1

print("Results for LinearSVC()")
print("Training time: %fs; Prediction time: %fs" % (time_liblinear_train, time_liblinear_predict))
print(classification_report(test_sentiment, prediction_liblinear))

Results for LinearSVC()
Training time: 0.681438s; Prediction time: 0.275039s
             precision    recall  f1-score   support

   negative       0.82      0.83      0.83      1000
   positive       0.83      0.81      0.82      1000

avg / total       0.82      0.82      0.82      2000



In [113]:
#SVM with kernel = linear 2 - TFIDF
classifier_liblinear = svm.LinearSVC()
t0=time.time()
classifier_liblinear.fit(CountV_train_t,train_df.sentiment)
t1=time.time()
prediction_liblinear = classifier_liblinear.predict(CountV_test_t)
t2=time.time()
time_liblinear_train = t1-t0
time_liblinear_predict = t2-t1

print("Results for LinearSVC() - TFIDF")
print("Training time: %fs; Prediction time: %fs" % (time_liblinear_train, time_liblinear_predict))
print(classification_report(test_sentiment, prediction_liblinear))

Results for LinearSVC() - TFIDF
Training time: 0.089778s; Prediction time: 0.014780s
             precision    recall  f1-score   support

   negative       0.84      0.86      0.85      1000
   positive       0.85      0.84      0.85      1000

avg / total       0.85      0.85      0.85      2000



In [105]:
#Gaussian Naive Bayes - CountVectorizer
from sklearn.naive_bayes import GaussianNB
classifier_gnb = GaussianNB()
t0=time.time()
classifier_gnb.fit(CountV_train,train_sentiment)
t1=time.time()
prediction_gnb = classifier_gnb.predict(CountV_test)
t2=time.time()
time_gnb_train=t1-t0
time_gnb_predict=t2-t1

print("Results for Gaussian Naive Bayes")
print("Training time: %fs; Prediction time: %fs" % (time_gnb_train, time_gnb_predict))
print(classification_report(test_sentiment, prediction_gnb))

Results for Gaussian Naive Bayes
Training time: 1.590973s; Prediction time: 1.693959s
             precision    recall  f1-score   support

   negative       0.58      0.65      0.61      1000
   positive       0.60      0.54      0.57      1000

avg / total       0.59      0.59      0.59      2000



In [114]:
#Gaussian Naive Bayes - TFIDF
from sklearn.naive_bayes import GaussianNB
classifier_gnb = GaussianNB()
t0=time.time()
classifier_gnb.fit(CountV_train_t,train_sentiment)
t1=time.time()
prediction_gnb = classifier_gnb.predict(CountV_test_t)
t2=time.time()
time_gnb_train=t1-t0
time_gnb_predict=t2-t1

print("Results for Gaussian Naive Bayes - TFIDF")
print("Training time: %fs; Prediction time: %fs" % (time_gnb_train, time_gnb_predict))
print(classification_report(test_sentiment, prediction_gnb))

Results for Gaussian Naive Bayes - TFIDF
Training time: 0.260416s; Prediction time: 0.286197s
             precision    recall  f1-score   support

   negative       0.65      0.73      0.69      1000
   positive       0.69      0.60      0.64      1000

avg / total       0.67      0.67      0.67      2000

